<a href="https://colab.research.google.com/github/dtht2d/bispectrum_component/blob/main/bispectrum/optimization/test_functions_calc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%autosave 30

Autosaving every 30 seconds


# **Code optimization:**  Compile function used in computing bispectrum components with/without Numba

**Numba**
- An open source JIT compiler that translates a subset of Python and NumPy code into fast machine code.
- Designed to be used with NumPy arrays and functions. Numba generates specialized code for different array data types and layouts to optimize performance.


In [2]:
pip install sympy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install numba

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
time: 4.25 s (started: 2023-02-28 19:07:20 +00:00)


In [ ]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 54.2 MB/s eta 0:00:00
time: 624 µs (started: 2023-02-28 19:07:20 +00:00)


## **a. Wigner D function**
Ref.[5], chapter 4.3-eq.1

$$
D^{j}_{mm'}(\theta_0,\theta,\phi)=e^{-im\theta_0}d^{j}_{mm'}(\theta)e^{-im'\phi}
$$
Choose 4.3.1 eq(4) to compute $d^{j}_{mm'}(\theta)$

\begin{align*}d^j_{mm'}{(\theta)}= [(j+m)!(j-m)!(j+m')!(j-m')!]^{\frac{1}{2}} \\ \times \displaystyle\sum_k(-1)^k\frac{(cos\frac{\theta}{2})^{2j-2k+m-m'}(sin\frac{\theta}{2})^{2k-m+m'}}{k!(j+m-k)!(j-m'-k)!(m'-m+k))} \end{align*}
**Note:** $k$ runs over all integer values for which factorial arguments are non-negative. The sums contain $(N+1)$  terms where N is the minimum of $j+m, j-m, j+m'$ and $j-m'$. 

**Finding** $k_{max}, k_{min}:$

\begin{align*} j+m-k &\geq 0 &\\ k &\leq j-m \leq j+m \\ j-m'-k &\geq 0 &\\ k &\leq j-m'  \\ k_{min}&= [\text{int}(\ j+m \ ), \text{int}(\ j-m' \ 
)] \\ m-m'+k&\geq 0 \\ k &\geq m'-m &\\ k_{max}&=\text{int}[0,m'-m] \end{align*} 


### **Compute class function without Numba**

In [ ]:
import numpy as np
import cmath

def fact(n):
    """
    This function is used to calculate factorial of a number by using
    an iterative approach instead of recursive approach
    """
    result = 1
    for i in range(1, n + 1):
        result *= i
    return result


class Wigner_D:
    """
    Args:
        j (scalar): angular momentum
        m (scalar): eigenvalue of angular momentum
        mp (scalar): eigenvalue of j along rotated axis
        theta_0 (scalar): first angle of rotation [0, pi]
        theta (scalar): second angle of rotation [0, pi]
        phi (scalar): third angle of rotation [0, 2*pi]
    Returns: complex number, Wigner D function
    ==========================Reference==================================
    [5] Chapter 4.3-(p.76,eq.1)  D.A. Varshalovich, A.N. Moskalev, V.K Khersonskii,
        Quantum Theory of Angular Momentum (1988)
    """
    def __init__(self, j, m, mp, theta_0, theta, phi):
        if j < 0 or not np.isclose(j, int(j)) or (j % 1 == 0.5 and (m % 1 != 0 or mp % 1 != 0)):
            raise ValueError("Invalid input parameters: j must be a non-negative integer or half-integer, "
                             "m and mp must be between -j and j.")
        if theta_0 < 0 or theta_0 > np.pi or theta < 0 or theta > np.pi or phi < 0 or phi > 2 * np.pi:
            raise ValueError(
                "Invalid input parameters: theta_0, theta, and phi must be within [0, pi] and [0, 2pi], respectively.")
        self.j = j
        self.m = m
        self.mp = mp
        self.theta_0 = theta_0
        self.theta = theta
        self.phi = phi
    def compute_dsmall(self):
        """
        This method is used to calculate the Wigner d small- real function involving trigonometric functions
        ==========================Reference==================================
        [5] Chapter 4.3.1-(p.76,eq.4)  D.A. Varshalovich, A.N. Moskalev, V.K Khersonskii,
        Returns: Wigner d - real function
        """
        kmax = max(0, self.m - self.mp)
        kmin = min(self.j + self.m, self.j - self.mp)
        term1 = np.sqrt(fact(self.j + self.m) * fact(self.j - self.m) * fact(self.j + self.mp) * fact(self.j - self.mp))
        sum = 0
        for k in range(kmax, kmin + 1):
            numerator = (-1) ** k * (cmath.cos(self.theta / 2)) ** (2 * self.j - 2 * k + self.m - self.mp) * \
                        (cmath.sin(self.theta / 2)) ** (2 * k - self.m + self.mp)
            denominator = fact(k) * fact(self.j + self.m - k) * fact(self.j - self.mp - k) * fact(self.mp - self.m + k)
            sum += numerator / denominator
        return sum*term1

    def wigner_D(self):
        term1 = np.exp(-1j * self.m * self.theta_0)
        term2 = self.compute_dsmall()
        term3 = np.exp(-1j * self.mp * self.phi)
        result = term1 * term2 * term3
        return result



**Example**

In [ ]:
import numpy as np
j, m, mp, theta_0, theta,phi= 1, 1, 0, np.pi, np.pi/2, 0
# Calculate the Wigner D function using our function
WD = Wigner_D(j, m, mp, theta_0, theta,phi)
wd = WD.wigner_D()
print(wd)

(0.7071067811865476+8.659560562354934e-17j)
time: 16.6 ms (started: 2023-02-28 19:07:27 +00:00)
